In [1]:
import sys
sys.path.append('/home/TomKerby/Research/lit_review/lit_review')
import utils
import rag_utils as rag
from langchain_neo4j import Neo4jVector

sys.path.append('/home/TomKerby/Research/lit_review/configs')
# from rag_config import config
from new_llm_config import config
# from huggingface_rag import config

kg = utils.load_kg(config)
llm_adapter = rag.get_llm(config)
emb_adapter = rag.get_embeddings(config)

custom_query = """
MATCH (c:Chunk)
WITH DISTINCT c, vector.similarity.cosine(c.textEmbedding, $embedding) AS score
ORDER BY score DESC LIMIT $k
RETURN c.text AS text, score, {source: c.source, chunkId: c.chunkId} AS metadata
"""

chunk_vector = Neo4jVector.from_existing_index(
    emb_adapter.embeddings,
    graph=kg, 
    index_name=config["rag"]["index_name"],
    embedding_node_property=config["rag"]["embedding_node_property"],
    text_node_property=config["rag"]["text_node_property"],
    retrieval_query=custom_query,
)

In [2]:
neo4j_retriever = chunk_vector.as_retriever()

In [3]:
from langchain_core.runnables import (
    RunnableParallel,
    RunnablePassthrough,
)
from langchain_core.prompts import PromptTemplate
from langchain_ollama import ChatOllama

simple_rag_chain = (
    {"context": neo4j_retriever, "question": RunnablePassthrough()}
    | PromptTemplate.from_template("Answer: {context}\nQuestion: {question}")
    | ChatOllama(model="llama3.3")
)

In [4]:
simple_rag_chain.invoke("What is a diffusion model?").pretty_print()

================================== Ai Message ==================================

A diffusion model is a type of probabilistic generative model that progressively destructs data by injecting noise and then learns to reverse this process for sample generation. In other words, it works by:

1. Adding increasing amounts of random noise to the input data (called the "diffusion" process)
2. Learning to remove the noise from the data to generate new samples

The goal is to learn a model that can generate new data samples that are similar to the original data. Diffusion models have been shown to be effective in various applications, including computer vision, natural language processing, and more.

There are several types of diffusion models, including:

1. Denoising diffusion probabilistic models (DDPMs)
2. Score-based generative models (SGMs)
3. Stochastic differential equations (Score SDEs)

These models work under the same principle of diffusion, but differ in their specific formulations 

In [ ]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub
from langchain_community.retrievers import WikipediaRetriever
from langchain.tools import Tool, tool
from langchain.chat_models import ChatOllama

# 1. Use the chat-style React template
react_prompt = hub.pull("hwchase17/react-chat")

# 2. Your two lookup tools…
wiki = WikipediaRetriever()
neokg = neo4j_retriever  # however you defined it

tools = [
    Tool("neo4j_kg_semantic_search", lambda q: neokg.invoke(q), "…"),
    Tool("wikipedia_search", lambda q: wiki.invoke(q), "…"),
]

# 3. Add the “final answer” tool
@tool("final_answer", return_direct=True)
def final_answer(text: str) -> str:
    """This tool helps the model exit infinite loops and give a final answer."""
    help_text = "\nThought: Do I need to use a tool? No\nFinal Answer: "
    return help_text + text

tools.append(final_answer)

# 4. Create agent & executor
agent = create_react_agent(
    llm=ChatOllama(model="llama3.3", temperature=0),
    tools=tools,
    prompt=react_prompt,
)

executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,  # keep this to auto-retry minor formatting slips
)

# 5. Run
out = executor.invoke({"input": "Please search the knowledge graph and use it to define what a diffusion model is?", "chat_history": []})
print(out["output"])


In [ ]:
from langchain.agents import create_self_ask_with_search_agent, AgentExecutor
from langchain_community.retrievers import WikipediaRetriever
from langchain.tools import Tool, tool
from langchain.chat_models import ChatOllama

# 2. Your two lookup tools…
wiki = WikipediaRetriever()
neokg = neo4j_retriever  # however you defined it

tools = [
    Tool("Intermediate Answer", neokg.invoke, "A tool for querying a knowledge graph using RAG.")
]


# 4. Create agent & executor
agent = create_self_ask_with_search_agent(
    llm=ChatOllama(model="llama3.3", temperature=0),
    tools=tools,
    prompt=prompt
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke(
    {"input": "What is a diffusion model and how does it differ from a GAN?"}
)

In [ ]:
from langchain import hub
from langchain_core.prompts import PromptTemplate
from langchain.agents import create_self_ask_with_search_agent, AgentExecutor
from langchain_community.retrievers import WikipediaRetriever
from langchain.tools import Tool
from langchain.chat_models import ChatOllama

# 1. Tools (must be exactly one, named "Intermediate Answer")
wiki = WikipediaRetriever()
tools = [
    Tool(
        name="Intermediate Answer",
        func=wiki.invoke,
        description="Useful for answering subquestions via search"
    )
]

from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["input", "agent_scratchpad"],
    template="""
{agent_scratchpad}
Question: {input}

Think step-by-step:
- If you need extra information, output exactly:
  Follow up: <subquestion>
- Otherwise, output exactly:
  So the final answer is: <your answer>
"""
)


# 3. Agent and executor with error handling
agent = create_self_ask_with_search_agent(
    llm=ChatOllama(model="llama3.3", temperature=0),
    tools=tools,
    prompt=prompt
)
executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors='generate',
    early_stopping_method='generate',
)

# 4. Run
result = executor.invoke({"input": "Can you describe in detail what is a diffusion model and how does it differ from a GAN?"})
print(result["output"])

In [ ]:
result = executor.invoke({"input": "Who wrote the paper titled 'DiffMat: Latent diffusion models for image-guided material generation'"})
print(result["output"])

In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama
from langchain_community.retrievers import WikipediaRetriever
from langchain.tools import Tool
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", """
You are a question-answering assistant.  
You have access to two tools:  
  • wikipedia_search(query): retrieves facts from Wikipedia  
  • neo4j_kg_semantic_search(query): retrieves facts from our knowledge graph  

**Workflow:**  
1. Think whether you need to call a tool.  
2. If yes, emit a tool call.  
3. After retrieving info, answer the user's question directly.  
"""),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])



llm = ChatOllama(
    model="llama3.3",
    temperature=0,
    num_ctx=131072,
)

wiki = WikipediaRetriever()

tools = [
    Tool("neo4j_kg_semantic_search", lambda q: neo4j_retriever.invoke(q), "A tool for searching a knowledge graph containing academic papers relating to the query."),
    Tool("wikipedia_search", lambda q: wiki.invoke(q), "A tool for searching wikipedia."),
]

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


from langchain_core.messages import AIMessage, HumanMessage
agent_executor.invoke(
    {
        "input": "What is a diffusion model?",
        "chat_history": [],
    }
)



> Entering new AgentExecutor chain...

Invoking: `wikipedia_search` with `Diffusion model`


[Document(metadata={'title': 'Diffusion model', 'summary': 'In machine learning, diffusion models, also known as diffusion probabilistic models or score-based generative models, are a class of latent variable generative models. A diffusion model consists of three major components: the forward process, the reverse process, and the sampling procedure. The goal of diffusion models is to learn a diffusion process for a given dataset, such that the process can generate new elements that are distributed similarly as the original dataset. A diffusion model models data as generated by a diffusion process, whereby a new datum performs a random walk with drift through the space of all possible data. A trained diffusion model can be sampled in many ways, with different efficiency and quality.\nThere are various equivalent formalisms, including Markov chains, denoising diffusion probabilistic models, noi

{'input': 'What is a diffusion model?',
 'chat_history': [],
 'output': 'Un modelo de difusión es un tipo de modelo generativo de aprendizaje automático que se utiliza para generar imágenes o otros tipos de datos a partir de una descripción textual. Estos modelos están basados en la técnica de difusión, que implica agregar ruido a las imágenes y luego eliminarlo para generar nuevas imágenes. Los modelos de difusión han sido utilizados con éxito en tareas como la generación de imágenes, la inpainting, la superresolución y la traducción de imágenes. Algunos ejemplos de modelos de difusión son el modelo de difusión estable (Stable Diffusion) y el modelo de difusión latente (Latent Diffusion Model). Estos modelos han sido desarrollados por investigadores en universidades y empresas, y han sido publicados en conferencias y revistas científicas.'}

In [ ]:
agent_executor.invoke(
    {
        "input": "What is a diffusion model?",
        "chat_history": [],
    }
)